In [96]:
%load_ext autoreload
%autoreload 2 
import pandas as pd
import numpy as np
import pdfplumber
import re
import nltk
from urllib.parse import urljoin, quote_plus, quote, urlencode
from urllib.request import Request, urlopen
from io import StringIO, BytesIO
from housecredit import HouseCredit
from try_scrape_demand import DemandDeposit

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# importing bank and text , tokenize

In [105]:
bankinter = DemandDeposit('https://clientebancario.bportugal.pt/sites/default/files/precario/0269_/0269_PRE.pdf', [x for x in range(8,19)]).is_avaiable()

abanca = DemandDeposit('https://clientebancario.bportugal.pt/sites/default/files/precario/0170_/0170_PRE.pdf', [x for x in range(6,10)]).is_avaiable()

bai = DemandDeposit('https://clientebancario.bportugal.pt/sites/default/files/precario/0008_/0008_PRE.pdf', None).is_avaiable()

bic = DemandDeposit('https://clientebancario.bportugal.pt/sites/default/files/precario/0079_/0079_PRE_0.pdf',[x for x in range(7,10)]).is_avaiable()

ctt = DemandDeposit('https://clientebancario.bportugal.pt/sites/default/files/precario/0193_/0193_PRE.pdf', [6]).is_avaiable()

This main product is not avaiable for the following bank: https://clientebancario.bportugal.pt/sites/default/files/precario/0008_/0008_PRE.pdf


In [357]:
text_bankinter = bankinter.getting_text()
text_abanca = abanca.getting_text()
text_ctt = ctt.getting_text()
bic = DemandDeposit('https://clientebancario.bportugal.pt/sites/default/files/precario/0079_/0079_PRE_0.pdf',[x for x in range(7,14)])
text_bic = bic.getting_text()

In [358]:
sent_bankinter = bankinter.tokenize()
sent_abanca = abanca.tokenize()
sent_bic = bic.tokenize()
sent_ctt = ctt.tokenize()

## importing dictionary

In [359]:
house_credit_com = {'admin':['Comissões associadas a atos administrativos 4.1 Não realização da escritura',
                             'Alteração do local da escritura',
                             'Declarações de dívida',
                             'Mudança de regime de crédito',
                             'Declarações de dívida',
                             'Pedido de 2ª via de Caderneta Predial',
                             'Emissão de declarações não obrigatórias por lei',
                             'Emissão de 2ª vias de Declaração para efeitos de IRS – Urgente',
                             'Emissão de 2º vias de Declaração para efeitos de IRS',
                             'Emissão de 2ª vias de faturas',
                             'Declaração de Dívida para Fins Diversos',
                             'Declaração de Encargos com Prestações'],
                    'certificates':['Emolumentos do registo predial', 'registo predial',
                                    'Certidão permanente on-line'],
                    'debt_recovery':['Comissão de recuperação de valores em dívida', 'Prestação até 50.000 €',
                                    'Prestação > 50.000 €', 'Comissão de recuperação de valores em dívida',
                                    'Prestação > 50.000,00€', 'Prestação ≤ 50.000,00€'],
                    'displacement':['Comissão de deslocação', 'Até 100 Kms', '101 a 250 Kms', '> 250 Kms '],
                    'early_payment':['Comissão de reembolso antecipado parcial', 'Taxa fixa', 'Taxa variável', 
                                    'Taxa fixa', 'Comissão de reembolso antecipado total', 'Comissão de antecipação',
                                    '(pré.aviso 7 dias)', 'Comissão de compra antecipada', '(pré-aviso 10 dias)', 
                                    'Comissão de Reembolso Antecipado Parcial',
                                    'Comissão de reembolso antecipado total'],
                    'evaluation':['Avaliação', 'Imóvel residencial', 
                                 'Garagens e arrecadações não anexas ao imóvel residencial', 'Avaliação do Imóvel'],
                    'formalization':['Comissão de formalização', 'Formalização'],
                    'process':['Processo', 'Abertura de Processo',
                              'Desistência ou não conclusão do processo por motivos imputáveis ao cliente'],
                    'inspections':['Vistorias', 'em caso de construção ou realização de obras'],
                    'reanalysis':['Reanálise'],
                    'settlement':['Comissão de Liquidação de Prestação', 'Liquidação de Prestação'],
                    'solicitors_notary':['Emolumentos notariais', 'Solicitadoria', 'Notiário'],
                    'statements':['Emissão de extratos de conta de empréstimos liquidados', 'extrato', 'extratos',
                                  'extrato de conta', 'extrato mensal'],
                    'taxes':['Imposto do Selo sobre concessão de crédito', 'imposto', 'imposto de selo', 'impostos'],
                    'termination':['Cessação da posição contratual', 'cessação', 'rescisão', 'encerramento']
                    }

# starting with new class

##  values

In [360]:
def values(tokenized, dictionary):
    values = [x for x in dictionary.values()]
    lista ={}
    for commission in values:
        for value in commission:
            for ind,sentence in enumerate(tokenized):
                if value in sentence:
                    if '[0-9]{1-2},[0-9]{2}' in sentence:
                        if value in lista:
                            lista[value].append(sentence)
                        else:
                            lista[value]= [sentence]
                    else:
                        if value in lista:
                            lista[value].append(' '.join([sentence, tokenized[ind+1]]))
                        else:
                            lista[value]= [' '.join([sentence,tokenized[ind+1]])]
    return lista

In [361]:
 values_ctt = values(sent_ctt, house_credit_com )

In [362]:
len(values_ctt.keys()) , len(house_credit_com.keys())

(16, 15)

- **values ctt is working properly**

- now we try also with the other 

In [363]:
values_abanca = values(sent_abanca, house_credit_com )
values_bankinter = values(sent_bankinter, house_credit_com )
values_bic = values(sent_bic, house_credit_com )

## n_account

In [364]:
abanca_extra = 'Leasing Habitação'
bankinter_extra = ['Outras finalidades','Locação financeira imobiliária']
bic= 'too many'


#### importing functions:

In [365]:
def tokenize(text):
    
    if len(nltk.sent_tokenize(text)) < 15:
        text = text.replace('\n','. ')
        # text = len_sentences(text)
        text = nltk.sent_tokenize(text)
        return text
    text = text.replace('\n',' ')
    # text = len_sentences(text)
    text = nltk.sent_tokenize(text)
    return text

In [366]:
def n_account(text,sent):
        accounts = []
        finals = []
        search = 'Crédito à habitação e outros créditos hipotecário'
        search2 = 'Crédito / Particulares'
        tokenized = sent
        for sentence in tokenized:
            if 'Crédito' in sentence or 'Habitação' in sentence:
                finals.append(sentence)
            
        return finals
#         for sentence in tokenized:
#             contas = re.split('Crédito',sentence)
#             accounts.append(contas)
#         for account in accounts:
#             if len(account)>1:
#                 account = ['Crédito'+ x for x in account]
#                 finals.append(account)
       
#         return finals

In [367]:
abanca_raw_accounts = n_account(text_abanca, sent_abanca)
print('FINISH')
ctt_raw_accounts =  n_account(text_ctt, sent_ctt)
print('FINISH')
bic_raw_accounts =  n_account(text_bic, sent_bic)
print('FINISH')
bankinter_raw_accounts =  n_account(text_bankinter, sent_bankinter)

FINISH
FINISH
FINISH


In [368]:
len(abanca_raw_accounts)

7

In [369]:
len(ctt_raw_accounts)

2

In [370]:
bic_raw_accounts

['Crédito à habitação e outros créditos hipotecários Comissões Acresce  Outras  Euros  Valor  Em %   Imposto condições (Mín/Máx) Anual Crédito à Habitação com Garantia Hipotecária Nota 1  Habitação Própria / Habitação Própria (oferta sénior) / Troca de Banco / Habitação para Arrendamento / Crédito Intercalar Terrenos / Troca de Casa (Fora  de comercialização) Crédito à Habitação sem Garantia Hipotecária Crédito Intercalar Sinal Outros Créditos Hipotecários Multifinalidades Posterior/Isolado / Multifinalidades Saúde / Crédito Hipotecário Investimentos Diversos  Comissões iniciais 1.',
 'Comissão de processo Habitação Própria nan 260,00 nan IS 4,00% Notas 2, 9, 10 Habitação Própria (oferta sénior) nan 260,00 nan IS 4,00% Notas 2, 9, 10 Troca de Casa (Fora de comercialização) nan 260,00 nan IS 4,00% Notas 2, 9, 10 Troca de Banco  nan 260,00 nan IS 4,00% Notas 2, 9, 11 Habitação para Arrendamento nan 260,00 nan IS 4,00% Notas 2, 9, 12 Crédito intercalar terrenos nan 50,00 nan IS 4,00% Nota

- account name is working for all but not for bankinter!!

## names

In [385]:
def names(values):
    search = 'Crédito à habitação e outros créditos hipotecário'
    search2 = 'Crédito / Particulares'
    search3 ='Aplicável'
    regular = []
    if len(values) < 10:
        return values
    else:
        for value in values:
            value =value.replace('Comissões Acresce  Outras  Euros  Valor  Em %   Imposto condições (Mín/Máx) Anual', '')
            value = value.replace('Outros Créditos Hipotecários', '')
            if search not in value and search2 not in value and search3 not in value:
                regular.append(value)
    return regular
#         return values
    
#     words = []
#     for account in values:
#         for element in account:
#             words.append(element.split())
#             print(words)
        
#     names = []
#     for word in words:
# #         word = [w.strip().lower() for w in word]
# #         print(word)
        
#         if word[0] == 'Crédito' and len(word)>1:
#             names.append(word)
#     finals = []
#     for name in names:
#         finals.append(' '.join(name[:14]).replace(';','').replace('/d./d',''))
#     search = 'Crédito à habitação e outros créditos hipotecário'
#     search2 = 'Crédito / Particulares'
#     regular = []
#     for final in finals:
# #         print(final)
#         if search not in final:
#             if search2 not in final:
#                 regular.append(final)
            
#     return regular
# #         if start not in regular:
# #             regular.append(final)
# #     lista =[]
# #     for name in regular:
# #         single = ' '.join(name.split(" ")[:2])
# #         if single not in lista:
# #             lista.append(name)
#     return finals

In [386]:
# abanca_names = names(abanca_raw_accounts)
# bankinter_names=names(bankinter_raw_accounts)
bic_names=names(bic_raw_accounts)
# ctt_names = names(ctt_raw_accounts)

In [387]:
bic_names

['Comissão de processo Habitação Própria nan 260,00 nan IS 4,00% Notas 2, 9, 10 Habitação Própria (oferta sénior) nan 260,00 nan IS 4,00% Notas 2, 9, 10 Troca de Casa (Fora de comercialização) nan 260,00 nan IS 4,00% Notas 2, 9, 10 Troca de Banco  nan 260,00 nan IS 4,00% Notas 2, 9, 11 Habitação para Arrendamento nan 260,00 nan IS 4,00% Notas 2, 9, 12 Crédito intercalar terrenos nan 50,00 nan IS 4,00% Nota 2 Crédito Intercalar Sinal nan 50,00 nan IS 4,00% Nota 2 Multifinalidades posterior / isolado  nan 250,00 nan IS 4,00% Notas 2, 3 Multifinalidades Saúde  nan 250,00 nan IS 4,00% Nota 2 Crédito Hipotecário Investimentos Diversos nan 250,00 nan IS 4,00% Nota 2 2.',
 'Nota 1  CréditoHabitação:Contratosdecréditoparaaquisição,construçãoerealizaçãodeobrasemhabitaçãoprópriapermanente,secundária ou para arrendamento.',
 'Incluem-se a Troca de Casa (Fora de comercialização), a Troca de Banco , o Crédito IntercalarTerrenos e o CréditoIntercalarSinal.OutrosCréditosHipotecários:Contratosdecrédit

In [292]:
HouseCredit('https://clientebancario.bportugal.pt/sites/default/files/precario/0170_/0170_PRE.pdf', [x for x in range(6,10)]).output()

Ready to scrape!
cleaned the text
we got the sentences


['OPERAÇÕES DE CRÉDITO (PARTICULARES)                                                                                 ÍNDICE  2.1 Crédito à habitação e outros créditos hipotecários    Comissões  Acresce    Valor  Outras condições  Em %  Euros (Mín/Máx)  Imposto  anual  Produto: Crédito Habitação Regime Geral e Regime Deficientes.',
 'Caracterização do produto: Crédito à habitação com garantia hipotecária destinado à aquisição, construção e realização de obras em  habitação própria e permanente, secundária ou para arrendamento (no Regime Deficientes só habitação própria e permanente) ou à  aquisição ou manutenção de direitos de propriedade sobre terrenos ou edifícios já existentes ou projetados.',
 'OPERAÇÕES DE CRÉDITO (PARTICULARES)                                                                                      ÍNDICE  2.1 Crédito à habitação e outros créditos  Comissões  hipotecários (cont.)',
 'Acresce  Outras condições    Euros  Valor  Imposto  Em %  (Mín/Máx)  anual  Produto:

In [294]:
bankinter = HouseCredit('https://clientebancario.bportugal.pt/sites/default/files/precario/0269_/0269_PRE.pdf', [x for x in range(8,19)]).output()

Ready to scrape!
cleaned the text
we got the sentences


IndexError: list index out of range